In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tf.keras.models import Sequential
from tf.keras.layers import Dense
from tf.keras.layers import LSTM
from tf.keras.layers import Dropout
from tf.keras.optimizers import RMSprop
from tf.keras.utils import to_categorical
from tf.keras.preprocessing.sequence import pad_sequences

In [ ]:
filename = 'persuasion_jane_austen.txt'

In [ ]:
raw_text = open(filename, 'r', encoding='utf-8').read()

In [ ]:
raw_text = raw_text.lower()

In [ ]:
chars = sorted(list(set(raw_text)))

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
max_len = 100
dx = []
dy = []
for i in range(0, len(raw_text) - max_len, 1):
  dx.append(raw_text[i : i + max_len])
  dy.append(raw_text[i + max_len])

In [ ]:
X = []
for i in range(0, len(dx), 1):
  X.append([char_indices[a] for a in dx[i]])

In [ ]:
y = []
for i in range(0, len(dy), 1):
  y.append([char_indices[a] for a in dy[i]])

In [ ]:
y = [a[0] for a in y]

In [ ]:
X2 = X[0:50000]

In [ ]:
X2 = np.reshape(X2, (len(X2), max_len, 1))

In [ ]:
X2 = X2 / float(len(chars))

In [ ]:
y2 = to_categorical(y)

In [ ]:
y2 = y2[0:50000]

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(X2.shape[1], X2.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y2.shape[1], activation='softmax'))
optimizer = RMSprop(learning_rate=0.01, clipnorm=1)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 128)               66560     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 60)                7740      
                                                                 
Total params: 74,300
Trainable params: 74,300
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X2, y2, epochs=30, batch_size=128, verbose=0)

In [ ]:
new = 'He had always intended to visit him, though to the last always assuring his wife that he should not go; and till the evening after the visit was paid she had no knowledge of it. it was then disclosed in the following manner. observing his second daughter employed in trimming a hat, he suddenly addressed her with'

In [ ]:
new = new.lower()

In [ ]:
def generate_chars(seed, length, model, maxlen):
  sentence = seed
  for i in range(length):
    seed_list = []
    for a in range(0, len(sentence), 1):
      seed_list.append([char_indices[j] for j in sentence[a]])
    seed_list_padded = pad_sequences([seed_list], maxlen = maxlen, padding='pre')
    seed_list_padded = seed_list_padded / float(len(chars))
    pred = model.predict(seed_list_padded, verbose=0)[0]
    pred = np.argmax(pred)
    pred = indices_char[pred]
    sentence += pred
  return(sentence)

In [ ]:
generate_chars(seed=new, length=50, model=model, maxlen=max_len)

'he had always intended to visit him, though to the last always assuring his wife that he should not go; and till the evening after the visit was paid she had no knowledge of it. it was then disclosed in the following manner. observing his second daughter employed in trimming a hat, he suddenly addressed her with ann the ceninnen of her father saseinen and annen'

'that he should not go; and till the evening after the visit was paid \n she had no knowledge of it. it was then disclosed in the following manner. observing his second daughter employed in trimming a hat, he suddenly addressed her with ann the ceninnen of her father saseinen and annen'